In [1]:
COLAB = True

In [2]:
if COLAB:
    from google_drive_downloader import GoogleDriveDownloader as gdd
    gdd.download_file_from_google_drive(file_id='1mamOwllpoOilUHMrXhBrY9azktQMME3q',
                                        dest_path='./datos_taidy/ml_data.json')

In [3]:
"""
import os
import glob
import pyAudioAnalysis
from pyAudioAnalysis import audioBasicIO as aIO
#from pyAudioAnalysis import audioSegmentation as aS
from pyAudioAnalysis.audioSegmentation import train_hmm_from_directory, hmm_segmentation

"""

'\nimport os\nimport glob\nimport pyAudioAnalysis\nfrom pyAudioAnalysis import audioBasicIO as aIO\n#from pyAudioAnalysis import audioSegmentation as aS\nfrom pyAudioAnalysis.audioSegmentation import train_hmm_from_directory, hmm_segmentation\n\n'

In [4]:
!ls

audio.mp3  datos_taidy	sample_data


In [5]:
!ls datos_taidy

ml_data.json


In [6]:
import json

with open('./datos_taidy/ml_data.json') as f:
    data = json.load(f)

data[0].keys()

dict_keys(['id', 'created_at', 'audio', 'labels'])

In [7]:
len(data)

87

In [8]:
data[76]["labels"]

{'breaths': [{'end': 6.781626891126148, 'start': 6.335651427628322}],
 'end_number': [{'number': '30'}],
 'numbers': {'10': {'end': 2.799845959179182,
   'original_length': 12.117333333333333,
   'start': 2.5138616933487374},
  '7': {'end': 1.7859017439621498,
   'original_length': 12.117333333333333,
   'start': 1.575913297023711},
  'End (30)': {'end': 11.36898023715415,
   'original_length': 12.117333333333333,
   'start': 11.021393629311767},
  'Start (1)': {'end': 0.1639909776090664,
   'original_length': 12.117333333333333,
   'start': 1.3877787807814457e-17}},
 'others': [{'label': 'Spanish', 'name': 'language'},
  {'label': 'Male', 'name': 'gender'}],
 'start_number': [{'number': '1'}],
 'text': []}

In [9]:
from IPython.display import JSON, Audio
import base64

In [10]:
JSON(data[0])

<IPython.core.display.JSON object>

In [11]:
aux = base64.b64decode(data[76]['audio']['data'].encode('utf-8'))
Audio(aux)

In [12]:
data[0]["audio"]["s3"]

's3://smart-dyspnea-tagging/sources/16-01-2021_21:17:02-806145f6-ebae-43e3-848f-3d6b26d803bd-audio.mp3'

In [13]:
import librosa

In [14]:
# Ejemplo de cargar audio con librosa
path = './audio.mp3'
with open(path, 'wb') as f:
    f.write(base64.b64decode(data[76]['audio']['data'].encode('utf-8')))
y, sr = librosa.load(path)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [15]:
y.shape

(267188,)

In [16]:
sr

22050

In [17]:
import numpy as np
from scipy.signal import butter,filtfilt

from scipy.io.wavfile import read as wr
from scipy.io.wavfile import write as ww

def butter_lowpass_filter(data, cutoff, fs, order):
    nyq = fs/2
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

def normalizar(x, secs_ventana=2, fs=44100):
    x2 = x.copy().astype(float)
    samples_ventana = int(secs_ventana*fs)
    aux = max(np.abs(x[:samples_ventana]).max(), 1)
    x2[:samples_ventana] = x2[:samples_ventana] / aux
    
    for i in range(samples_ventana, len(x2)):
        aux = max(np.abs(x[(i+1-samples_ventana):(i+1)]).max(), 1)
        x2[i] = x2[i] / aux
    return x2

def preprocesa(fichero_orig, fichero_dest):
    Fs, x = wr(fichero_orig)
    
    if len(x.shape) > 1:
        x = x[:,0]
        
    #x2 = normalizar(x, secs_ventana=1, fs=Fs)
    x2 = normalizar(x, secs_ventana=0.5, fs=Fs) # este funciona bien
    #x2 = normalizar(x, secs_ventana=0.2, fs=Fs)
    #x2 = normalizar(x, secs_ventana=0.1, fs=Fs)

    #x2 = normalizar(butter_lowpass_filter(x, 7000, Fs, 10), secs_ventana=1, fs=Fs)

    x2 = x2 / np.abs(x2).max()
    x2 = x2 * 30000 # / 4 / 4
    x2 = x2.astype(np.int16)
    #x3 = np.zeros((len(x2),2), dtype=np.int16)
    #x3[:,0] = x2; x3[:,1] = x2

    ww(fichero_dest, Fs, x2)